In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
import dask.dataframe as dd
from tqdm.notebook import tqdm
import os
import time
import modin.pandas as mpd

# 경고 무시 모듈
import warnings

# 경고를 무시
warnings.filterwarnings(action='ignore')

# 대용량 파일 확인용

In [2]:
txt = './원본/Wi-Fi/presence_full.txt'

In [3]:
dask_df = dd.read_csv(txt)

In [5]:
df = dask_df.head(20)

In [6]:
df

CLIENTMAC         RADIO  STARTTIMESTAMP  ENDTIMESTAMP
0   5077050CAD17  7C573C745F50      1588062600    1588063200
1   5077050CC3C1  7C573C746C90      1588062600    1588063200
2   5077050D7171  7C573C743990      1588062600    1588063200
3   5077050DF015  7C573C74D100      1588062600    1588063200
4   507705103E19  7C573C7454F0      1588062600    1588063200
5   5077050FE81D  7C573C743990      1588062600    1588063200
6   5077051140EF  7C573C7454F0      1588062600    1588063200
7   507705114215  7C573C746C90      1588062600    1588063200
8   50770513672B  7C573C7456F0      1588062600    1588063200
9   50770518494D  7C573C74CE90      1588062600    1588063200
10  5077051AA38B  7C573C74E550      1588062600    1588063200
11  5077051BC1C7  7C573C7454F0      1588062600    1588063200
12  5077051CF65D  7C573C74B810      1588062600    1588063200
13  5077051D9767  7C573C74B360      1588062600    1588063200
14  50770524155D  7C573C742BF0      1588062600    1588063200
15  5077052608A5  7C573C74B2F0      1588062600    1588063200
16  5077052611FF  7C573C7455A0      1588062600    1588063200
17  507705283961  7C573C74BD50      1588062600    1588063200
18  507705285DFB  7C573C742BF0      1588062600    1588063200
19  5077052C473B  7C573C74D110      1588062600    1588063200

# -------------------------------------------------------------------------------

# 테마형특화단지 AP

In [ ]:
# 테마형특화단지 MAC Address 대문자로 변경
df = pd.read_csv('./원본/테마형특화단지ap상세/테마형특화단지ap.csv', encoding='cp949')

for n in range(len(df)):
    df['MAC주소'][n] = df['MAC주소'][n].upper()

df.drop('순번', axis=1, inplace=True)
df = df[['AP명','MAC주소']]
df.to_csv('./원본/테마형특화단지ap상세/ap_upper.csv',encoding='utf-8', index=False)

In [16]:
ap_upper = dd.read_csv('./원본/테마형특화단지ap상세/ap_upper.csv')

In [17]:
ap_upper = ap_upper.head(len(ap_upper))

In [18]:
ap_upper

AP명         MAC주소
0           Park_yeonji_clocktower  7C573CCF4526
1           Park_yeonji_windpole01  7C573CCF4A7E
2           Park_yeonji_windpole02  7C573CCF4D6A
3           Park_yeonji_38parklamp  7C573CCF45F4
4            Park_yeonji_jogakpark  7C573CCF49C0
..                             ...           ...
69  Street_emartexit_trafficpole02  7C573CCF4BD4
70   Street_passengerterminal_CCTV  7C573CCF48D4
71       Station_bonghwang_3exit01  7C573CCF4E54
72       Station_bonghwang_3exit02  7C573CCF4AE6
73         Terminal_Shinsegae_bike  7C573CCFB76A

[74 rows x 2 columns]

# -------------------------------------------------------------------------------

# AP

In [9]:
ap_txt = './원본/Wi-Fi/ap.txt'

In [11]:
%%time
# 불러오는 시간
# CPU times: total: 28.1 s
# Wall time: 28.1 s
ap = pd.read_csv(ap_txt)

CPU times: total: 28.1 s
Wall time: 28.1 s


In [13]:
print('데이터 Row :',len(ap))

데이터 Row : 36575274


In [14]:
ap.head(1)

APMAC  APNAME STARTTIMESTAMP ENDTIMESTAMP         BSSID  \
0  7C573CCF4C04  AP_053     1588137600   1588138200  7C573C74C050   

          RADIO APIP  
0  7C573C74C050  NaN

In [26]:
# 테마형 특화단지 MAC Address 추출
ap_up_list = list(ap_upper['MAC주소'])
print('Wi-Fi 개수(MAC Address) :', len(ap_up_list), '개소')

Wi-Fi 개수(MAC Address) : 74 개소


In [25]:
# MAC Address 중복값 제거
ap_np_set = list(set(ap['APMAC']))
print('AP MAC :', len(ap_np_set),'개')

AP MAC : 222 개


In [32]:
# AP 데이터프레임에 테마형특화단지 Wi-Fi의
# 모든 개소가 있는지 확인
a = []
for n in ap_up_list:
    if n in ap_np_set:
        a.append(n)
    else:
        print(n, '- 없음')
if len(a) == len(ap_up_list):
    print('ALL in')
else:
    print(len(ap_up_list) - len(a), '개 부족')

ALL in


In [75]:
# 필요한 MAC Address를 찾기위한 표현식 자동 작성
str_list = None
for n in range(len(a)):
    if n == 0:
        str_list = a[n]
    else:
        str_list += ('|'+ a[n])

In [76]:
str_list

'7C573CCF4526|7C573CCF4A7E|7C573CCF4D6A|7C573CCF45F4|7C573CCF49C0|7C573CCF4C3E|7C573CCF4AEE|7C573CCF43DE|7C573CCF44C0|7C573CCF4740|7C573CCF46C8|7C573CCF4D0C|7C573CCF4E8C|7C573CCF4D74|7C573CCF4398|7C573CCF42BE|7C573CCF4518|7C573CCF4D10|7C573CCF47E6|7C573CCF454E|7C573CCF465E|7C573CCF457A|7C573CCF4342|7C573CCF497E|7C573CCF4CCC|7C573CCF456E|7C573CCF45BA|7C573CCF43EE|7C573CCF455A|7C573CCF44DA|7C573CCF470A|7C573CCF49AA|7C573CCF4A6E|7C573CCF4BDA|7C573CCF4CE8|7C573CCF4352|7C573CCF4F88|7C573CCF4A90|7C573CCF4320|7C573CCF4AE0|7C573CCF4AD6|7C573CCF4B80|7C573CCF4E28|7C573CCF4CF0|7C573CCF4F1E|7C573CCF484E|7C573CCF4ECC|7C573CCF4AC0|7C573CCF4B2E|7C573CCF49F8|7C573CCF4D54|7C573CCF4A94|7C573CCF4F62|7C573CCF4EA6|7C573CCF4302|D015A6C7F506|D015A6C7F4F2|7C573CCF4AAE|7C573CCF4B7C|7C573CCF47D0|7C573CCF4D94|7C573CCF4C04|7C573CCF4B36|7C573CCF4AF2|7C573CCF4AFE|7C573CCF4EF2|7C573CCF4B98|7C573CCF471C|7C573CCF4F78|7C573CCF4BD4|7C573CCF48D4|7C573CCF4E54|7C573CCF4AE6|7C573CCFB76A'

In [78]:
ap_use = ap[ap['APMAC'].str.contains(str_list)]

In [98]:
ap_use.reset_index(drop=True, inplace=True)

In [99]:
ap_use

APMAC                   APNAME STARTTIMESTAMP ENDTIMESTAMP  \
0         7C573CCF4C04                   AP_053     1588137600   1588138200   
1         7C573CCF4C04                   AP_053     1588137600   1588138200   
2         7C573CCF4CE8                   AP_072     1588137600   1588138200   
3         7C573CCF4CE8                   AP_072     1588137600   1588138200   
4         7C573CCF4CF0                   AP_034     1588137600   1588138200   
...                ...                      ...            ...          ...   
15368316  7C573CCFB76A  Terminal_shinsegae_bike     1669359600   1669360200   
15368317  7C573CCFB76A  Terminal_shinsegae_bike     1669546200   1669546800   
15368318  7C573CCFB76A  Terminal_shinsegae_bike     1669546200   1669546800   
15368319  D015A6C7F4F2  Tomb_kingsuro_infobox02     1669546200   1669546800   
15368320  D015A6C7F4F2  Tomb_kingsuro_infobox02     1669546200   1669546800   

                 BSSID         RADIO  \
0         7C573C74C050  7C573C74C050   
1         7C573C74C040  7C573C74C040   
2         7C573C74CE90  7C573C74CE92   
3         7C573C74CE80  7C573C74CE80   
4         7C573C74CF10  7C573C74CF12   
...                ...           ...   
15368316  7C573C7B76A0  7C573C7B76A0   
15368317  7C573C7B76B0  7C573C7B76B0   
15368318  7C573C7B76A0  7C573C7B76A0   
15368319  D015A6FF4F30  D015A6FF4F30   
15368320  D015A6FF4F20  D015A6FF4F20   

                                                       APIP  
0                                                       NaN  
1                                                       NaN  
2                                                       NaN  
3                                                       NaN  
4                                                       NaN  
...                                                     ...  
15368316                                      192.168.90.42  
15368317                                      192.168.90.42  
15368318                                      192.168.90.42  
15368319                                      192.168.90.22  
15368320  192.168.90.22                                 ...  

[15368321 rows x 7 columns]

In [100]:
ap_use.to_csv('./전처리/wi-fi/ap.csv', encoding='utf-8', index=False) 

# AP - Unix time 변환

In [33]:
ap_use = pd.read_csv('./전처리/wi-fi/ap.csv', encoding='utf-8')

In [34]:
ap_use.head()

APMAC  APNAME       STARTTIMESTAMP         ENDTIMESTAMP  \
0  7C573CCF4C04  AP_053  2020-04-29 14:20:00  2020-04-29 14:30:00   
1  7C573CCF4C04  AP_053  2020-04-29 14:20:00  2020-04-29 14:30:00   
2  7C573CCF4CE8  AP_072  2020-04-29 14:20:00  2020-04-29 14:30:00   
3  7C573CCF4CE8  AP_072  2020-04-29 14:20:00  2020-04-29 14:30:00   
4  7C573CCF4CF0  AP_034  2020-04-29 14:20:00  2020-04-29 14:30:00   

          BSSID         RADIO APIP  
0  7C573C74C050  7C573C74C050  NaN  
1  7C573C74C040  7C573C74C040  NaN  
2  7C573C74CE90  7C573C74CE92  NaN  
3  7C573C74CE80  7C573C74CE80  NaN  
4  7C573C74CF10  7C573C74CF12  NaN

In [35]:
ap_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15368321 entries, 0 to 15368320
Data columns (total 7 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   APMAC           object
 1   APNAME          object
 2   STARTTIMESTAMP  object
 3   ENDTIMESTAMP    object
 4   BSSID           object
 5   RADIO           object
 6   APIP            object
dtypes: object(7)
memory usage: 820.8+ MB


In [113]:
for n in tqdm(range(len(ap_use))):
    ap_use['STARTTIMESTAMP'][n] = datetime.fromtimestamp(ap_use['STARTTIMESTAMP'][n])
    ap_use['ENDTIMESTAMP'][n] = datetime.fromtimestamp(ap_use['ENDTIMESTAMP'][n])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17796\1885891549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_use['STARTTIMESTAMP'][n] = datetime.fromtimestamp(ap_use['STARTTIMESTAMP'][n])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17796\1885891549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_use['ENDTIMESTAMP'][n] = datetime.fromtimestamp(ap_use['ENDTIMESTAMP'][n])


In [114]:
ap_use

APMAC                   APNAME       STARTTIMESTAMP  \
0         7C573CCF4C04                   AP_053  2020-04-29 14:20:00   
1         7C573CCF4C04                   AP_053  2020-04-29 14:20:00   
2         7C573CCF4CE8                   AP_072  2020-04-29 14:20:00   
3         7C573CCF4CE8                   AP_072  2020-04-29 14:20:00   
4         7C573CCF4CF0                   AP_034  2020-04-29 14:20:00   
...                ...                      ...                  ...   
15368316  7C573CCFB76A  Terminal_shinsegae_bike  2022-11-25 16:00:00   
15368317  7C573CCFB76A  Terminal_shinsegae_bike  2022-11-27 19:50:00   
15368318  7C573CCFB76A  Terminal_shinsegae_bike  2022-11-27 19:50:00   
15368319  D015A6C7F4F2  Tomb_kingsuro_infobox02  2022-11-27 19:50:00   
15368320  D015A6C7F4F2  Tomb_kingsuro_infobox02  2022-11-27 19:50:00   

                 ENDTIMESTAMP         BSSID         RADIO  \
0         2020-04-29 14:30:00  7C573C74C050  7C573C74C050   
1         2020-04-29 14:30:00  7C573C74C040  7C573C74C040   
2         2020-04-29 14:30:00  7C573C74CE90  7C573C74CE92   
3         2020-04-29 14:30:00  7C573C74CE80  7C573C74CE80   
4         2020-04-29 14:30:00  7C573C74CF10  7C573C74CF12   
...                       ...           ...           ...   
15368316  2022-11-25 16:10:00  7C573C7B76A0  7C573C7B76A0   
15368317  2022-11-27 20:00:00  7C573C7B76B0  7C573C7B76B0   
15368318  2022-11-27 20:00:00  7C573C7B76A0  7C573C7B76A0   
15368319  2022-11-27 20:00:00  D015A6FF4F30  D015A6FF4F30   
15368320  2022-11-27 20:00:00  D015A6FF4F20  D015A6FF4F20   

                                                       APIP  
0                                                       NaN  
1                                                       NaN  
2                                                       NaN  
3                                                       NaN  
4                                                       NaN  
...                                                     ...  
15368316                                      192.168.90.42  
15368317                                      192.168.90.42  
15368318                                      192.168.90.42  
15368319                                      192.168.90.22  
15368320  192.168.90.22                                 ...  

[15368321 rows x 7 columns]

In [118]:
ap_use.to_csv('./전처리/wi-fi/ap.csv', encoding='utf-8', index=False)

# AP - BSSID & RADIO 유일값 확인

In [36]:
ap = pd.read_csv('./전처리/wi-fi/ap.csv', encoding='utf-8')

In [37]:
ap.head()

APMAC  APNAME       STARTTIMESTAMP         ENDTIMESTAMP  \
0  7C573CCF4C04  AP_053  2020-04-29 14:20:00  2020-04-29 14:30:00   
1  7C573CCF4C04  AP_053  2020-04-29 14:20:00  2020-04-29 14:30:00   
2  7C573CCF4CE8  AP_072  2020-04-29 14:20:00  2020-04-29 14:30:00   
3  7C573CCF4CE8  AP_072  2020-04-29 14:20:00  2020-04-29 14:30:00   
4  7C573CCF4CF0  AP_034  2020-04-29 14:20:00  2020-04-29 14:30:00   

          BSSID         RADIO APIP  
0  7C573C74C050  7C573C74C050  NaN  
1  7C573C74C040  7C573C74C040  NaN  
2  7C573C74CE90  7C573C74CE92  NaN  
3  7C573C74CE80  7C573C74CE80  NaN  
4  7C573C74CF10  7C573C74CF12  NaN

In [65]:
BSSID = list(set(ap['BSSID']))
RADIO = list(set(ap['RADIO']))
APMAC = list(set(ap['APMAC']))

In [66]:
print('BSSID :' ,len(BSSID))
print('RADIO :' ,len(RADIO))
print('APMAC :' ,len(APMAC))

BSSID : 148
RADIO : 148
APMAC : 74


### 결과 : 1개의  BSSID, RADIO에 1개의 APMAC이 매칭됨

In [67]:
b_am = []
bi= []
for n in tqdm(BSSID):
    li = list((set(ap[ap['BSSID'] == n]['APMAC'])))
    if len(li) != 1:
        print(li)
    else:
        b_am.append(li[0])
        bi.append(n)
print('done')

100%|████████████████████████████████████████████████████████████████████████████████| 148/148 [01:48<00:00,  1.37it/s]

done


In [68]:
r_am = []
rd = []
for n in tqdm(RADIO):
    li = list((set(ap[ap['RADIO'] == n]['APMAC'])))
    
    if len(li) != 1:
        print(li)
    else:
        r_am.append(li[0])
        rd.append(n)
print('done')

100%|████████████████████████████████████████████████████████████████████████████████| 148/148 [01:45<00:00,  1.40it/s]

done


### 결과 : 1개의 APMAC에  BSSID, RADIO 각각 2개씩 매칭됨

In [123]:
for n in tqdm(APMAC):
    li = list((set(ap[ap['APMAC'] == n]['BSSID'])))
    if len(li) != 2:
        print(n, li)
print('done')

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:56<00:00,  1.31it/s]

done


In [124]:
for n in tqdm(APMAC):
    li = list((set(ap[ap['APMAC'] == n]['RADIO'])))
    if len(li) != 2:
        print(n, li)
print('done')

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:54<00:00,  1.36it/s]

done


In [69]:
RADIO_dict = {'APMAC' : r_am, 'RADIO':rd}
RADIO_df = pd.DataFrame(RADIO_dict)

In [70]:
RADIO_df.head()

APMAC         RADIO
0  7C573CCF4EF2  7C573C74EF30
1  7C573CCF456E  7C573C7456E0
2  7C573CCF4D6A  7C573C74D6B0
3  7C573CCF4F88  7C573C74F880
4  7C573CCF4320  7C573C743200

In [71]:
RADIO_df.sort_values('APMAC', inplace=True)
RADIO_df.reset_index(drop=True, inplace=True)
RADIO_df.head()

APMAC         RADIO
0  7C573CCF42BE  7C573C742BF0
1  7C573CCF42BE  7C573C742BE0
2  7C573CCF4302  7C573C743030
3  7C573CCF4302  7C573C743020
4  7C573CCF4320  7C573C743200

In [72]:
BSSID_dict = {'APMAC' : b_am, 'BSSID':bi}
BSSID_df = pd.DataFrame(BSSID_dict)
BSSID_df.head()

APMAC         BSSID
0  7C573CCF4EF2  7C573C74EF30
1  7C573CCF456E  7C573C7456E0
2  7C573CCF4D6A  7C573C74D6B0
3  7C573CCF4F88  7C573C74F880
4  7C573CCF4320  7C573C743200

In [73]:
BSSID_df.sort_values('APMAC', inplace=True)
BSSID_df.reset_index(drop=True, inplace=True)
BSSID_df.head()

APMAC         BSSID
0  7C573CCF42BE  7C573C742BE0
1  7C573CCF42BE  7C573C742BF0
2  7C573CCF4302  7C573C743030
3  7C573CCF4302  7C573C743020
4  7C573CCF4320  7C573C743210

In [74]:
MAC_df = pd.concat([BSSID_df,RADIO_df.drop('APMAC', axis=1)], axis=1)
MAC_df

APMAC         BSSID         RADIO
0    7C573CCF42BE  7C573C742BE0  7C573C742BF0
1    7C573CCF42BE  7C573C742BF0  7C573C742BE0
2    7C573CCF4302  7C573C743030  7C573C743030
3    7C573CCF4302  7C573C743020  7C573C743020
4    7C573CCF4320  7C573C743210  7C573C743200
..            ...           ...           ...
143  7C573CCFB76A  7C573C7B76A0  7C573C7B76B0
144  D015A6C7F4F2  D015A6FF4F20  D015A6FF4F30
145  D015A6C7F4F2  D015A6FF4F30  D015A6FF4F20
146  D015A6C7F506  D015A6FF5060  D015A6FF5060
147  D015A6C7F506  D015A6FF5070  D015A6FF5070

[148 rows x 3 columns]

In [143]:
#MAC_df.to_excel('./전처리/wi-fi/MAC_BSSID_RADIO_매칭.xlsx', encoding='cp949', index=False)

In [75]:
MAC_df.to_csv('./전처리/wi-fi/MAC_BSSID_RADIO_매칭.csv', encoding='cp949', index=False)

# BSSID & RADIO 추출

In [80]:
BSSID = list(set(ap_use['BSSID']))
RADIO = list(set(ap_use['RADIO']))

In [81]:
BSSID

['7C573C74A900',
 '7C573C74BD50',
 '7C573C7465F0',
 '7C573C74AE10',
 '7C573C74F630',
 '7C573C74D0C0',
 '7C573C74D0D0',
 '7C573C7465E0',
 '7C573C74ECC0',
 '7C573C74AAF0',
 '7C573C744DA0',
 '7C573C7455B0',
 '7C573C74B7C0',
 '7C573C74D750',
 '7C573C74D550',
 '7C573C748D50',
 '7C573C7470B0',
 '7C573C747E60',
 '7C573C74A7F0',
 '7C573C7455A0',
 '7C573C74AAE0',
 '7C573C74B360',
 '7C573C74EA70',
 '7C573C747D10',
 '7C573C745270',
 '7C573C74C040',
 '7C573C74AEE0',
 '7C573C7484F0',
 '7C573C743420',
 '7C573C746C80',
 '7C573C74B990',
 '7C573C744C10',
 '7C573C74A940',
 '7C573C74CE80',
 '7C573C74CF10',
 '7C573C749C00',
 '7C573C74B370',
 '7C573C74AEF0',
 '7C573C743210',
 '7C573C74EA60',
 '7C573C7457B0',
 '7C573C743030',
 '7C573C745190',
 '7C573C74AE60',
 '7C573C74C3E0',
 '7C573C74AF20',
 '7C573C74CCC0',
 '7C573C74A6E0',
 '7C573C745180',
 '7C573C743520',
 '7C573C746C90',
 '7C573C74C3F0',
 '7C573C743530',
 '7C573C744C00',
 '7C573C742BF0',
 '7C573C74D950',
 '7C573C743980',
 '7C573C7497F0',
 '7C573C74D100

In [82]:
RADIO

['7C573C74A900',
 '7C573C74B992',
 '7C573C74BD50',
 '7C573C7465F0',
 '7C573C74AE10',
 '7C573C74F630',
 '7C573C74D0C0',
 '7C573C74D0D0',
 '7C573C7465E0',
 '7C573C74ECC0',
 '7C573C74AAF0',
 '7C573C744DA0',
 '7C573C74AF32',
 '7C573C7455B0',
 '7C573C74B7C0',
 '7C573C74AFE2',
 '7C573C74D750',
 '7C573C743532',
 '7C573C74D550',
 '7C573C748D50',
 '7C573C7470B0',
 '7C573C7471D2',
 '7C573C747E60',
 '7C573C7455A0',
 '7C573C74A7F0',
 '7C573C74AAE0',
 '7C573C74EA70',
 '7C573C74B360',
 '7C573C747D10',
 '7C573C745270',
 '7C573C74C040',
 '7C573C74AEE0',
 '7C573C743420',
 '7C573C746C80',
 '7C573C744C10',
 '7C573C74A940',
 '7C573C74CE80',
 '7C573C749C00',
 '7C573C74B370',
 '7C573C74AEF0',
 '7C573C743210',
 '7C573C74EA60',
 '7C573C7457B0',
 '7C573C743030',
 '7C573C745190',
 '7C573C74AE60',
 '7C573C74C3E0',
 '7C573C74CCC0',
 '7C573C74A6E0',
 '7C573C745180',
 '7C573C743520',
 '7C573C746C90',
 '7C573C74C3F0',
 '7C573C74CE92',
 '7C573C7484F2',
 '7C573C744C00',
 '7C573C74B812',
 '7C573C742BF0',
 '7C573C74D950

# -------------------------------------------------------------------------------

# station

In [91]:
station = pd.read_csv('./원본/Wi-Fi/station.txt')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_15004\433213972.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  station = pd.read_csv('C:/Users/user/Desktop/와이파이/Desktop/station.txt')


In [92]:
station

CLIENTMAC         BSSID STARTTIMESTAMP ENDTIMESTAMP
0         940C98B56271  7C573C747400     1588141200   1588141800
1         B85A7315DBCB  484AE9D971D0     1588141200   1588141800
2         C0E86238BFBE  484AE9D9C9C0     1588141200   1588141800
3         9476B76FB07F  484AE9D99B50     1588141200   1588141800
4         C4618BA459F9  484AE9DA0CB0     1588141200   1588141800
...                ...           ...            ...          ...
14580812  2418C627ADD7  7C573C7454F0     1669475400   1669476000
14580813  4E6623717148  7C573C77FBB0     1669475400   1669476000
14580814  8649CA41BA1F  484AE9D9C9D0     1669475400   1669476000
14580815  A2FDD95F5C52  7C573C780E90     1669475400   1669476000
14580816  3AE1AAF05270  7C573C77FBB0     1669475400   1669476000

[14580817 rows x 4 columns]

In [93]:
station_str_list = None
for n in range(len(BSSID)):
    if n == 0:
        station_str_list = BSSID[n]
    else:
        station_str_list += ('|'+ BSSID[n])

In [94]:
station_str_list

'7C573C74A900|7C573C74BD50|7C573C7465F0|7C573C74AE10|7C573C74F630|7C573C74D0C0|7C573C74D0D0|7C573C7465E0|7C573C74ECC0|7C573C74AAF0|7C573C744DA0|7C573C7455B0|7C573C74B7C0|7C573C74D750|7C573C74D550|7C573C748D50|7C573C7470B0|7C573C747E60|7C573C74A7F0|7C573C7455A0|7C573C74AAE0|7C573C74B360|7C573C74EA70|7C573C747D10|7C573C745270|7C573C74C040|7C573C74AEE0|7C573C7484F0|7C573C743420|7C573C746C80|7C573C74B990|7C573C744C10|7C573C74A940|7C573C74CE80|7C573C74CF10|7C573C749C00|7C573C74B370|7C573C74AEF0|7C573C743210|7C573C74EA60|7C573C7457B0|7C573C743030|7C573C745190|7C573C74AE60|7C573C74C3E0|7C573C74AF20|7C573C74CCC0|7C573C74A6E0|7C573C745180|7C573C743520|7C573C746C90|7C573C74C3F0|7C573C743530|7C573C744C00|7C573C742BF0|7C573C74D950|7C573C743980|7C573C7497F0|7C573C74D100|7C573C74E8D0|7C573C745F40|7C573C747400|7C573C749AB0|7C573C74BD40|7C573C74CF00|7C573C74B7D0|7C573C74CE90|7C573C74AD70|7C573C74E280|7C573C74D940|7C573C74E8C0|7C573C7456F0|7C573C74C050|7C573C743EE0|7C573C74ECD0|7C573C74B2F0|7C573C749F8

In [95]:
station_use = station[station['BSSID'].str.contains(station_str_list)]
station_use

CLIENTMAC         BSSID STARTTIMESTAMP ENDTIMESTAMP
0         940C98B56271  7C573C747400     1588141200   1588141800
7         9476B78E5BB3  7C573C746C80     1588141200   1588141800
9         948BC111791C  7C573C7457B0     1588141200   1588141800
10        948BC1006C66  7C573C743030     1588141200   1588141800
11        9476B7B12657  7C573C74B7C0     1588141200   1588141800
...                ...           ...            ...          ...
14580795  A674B0AB737C  7C573C7455A0     1669475400   1669476000
14580802  8A4610BA5CC4  7C573C7497F0     1669475400   1669476000
14580809  6E4DE27A1B9B  7C573C7470B0     1669475400   1669476000
14580811  04B1679C7257  7C573C74B2E0     1669475400   1669476000
14580812  2418C627ADD7  7C573C7454F0     1669475400   1669476000

[8342652 rows x 4 columns]

In [96]:
station_use.reset_index(drop=True, inplace=True)

In [97]:
station_use

CLIENTMAC         BSSID STARTTIMESTAMP ENDTIMESTAMP
0        940C98B56271  7C573C747400     1588141200   1588141800
1        9476B78E5BB3  7C573C746C80     1588141200   1588141800
2        948BC111791C  7C573C7457B0     1588141200   1588141800
3        948BC1006C66  7C573C743030     1588141200   1588141800
4        9476B7B12657  7C573C74B7C0     1588141200   1588141800
...               ...           ...            ...          ...
8342647  A674B0AB737C  7C573C7455A0     1669475400   1669476000
8342648  8A4610BA5CC4  7C573C7497F0     1669475400   1669476000
8342649  6E4DE27A1B9B  7C573C7470B0     1669475400   1669476000
8342650  04B1679C7257  7C573C74B2E0     1669475400   1669476000
8342651  2418C627ADD7  7C573C7454F0     1669475400   1669476000

[8342652 rows x 4 columns]

In [101]:
station_use.to_csv('./전처리/wi-fi/station.csv', encoding='utf-8', index=False)

# station - Unix time 변환

In [140]:
station_use = pd.read_csv('./전처리/wi-fi/station.csv', encoding='utf-8')

In [141]:
station_use

CLIENTMAC         BSSID  STARTTIMESTAMP  ENDTIMESTAMP
0        940C98B56271  7C573C747400      1588141200    1588141800
1        9476B78E5BB3  7C573C746C80      1588141200    1588141800
2        948BC111791C  7C573C7457B0      1588141200    1588141800
3        948BC1006C66  7C573C743030      1588141200    1588141800
4        9476B7B12657  7C573C74B7C0      1588141200    1588141800
...               ...           ...             ...           ...
8342647  A674B0AB737C  7C573C7455A0      1669475400    1669476000
8342648  8A4610BA5CC4  7C573C7497F0      1669475400    1669476000
8342649  6E4DE27A1B9B  7C573C7470B0      1669475400    1669476000
8342650  04B1679C7257  7C573C74B2E0      1669475400    1669476000
8342651  2418C627ADD7  7C573C7454F0      1669475400    1669476000

[8342652 rows x 4 columns]

In [142]:
station_use.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8342652 entries, 0 to 8342651
Data columns (total 4 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   CLIENTMAC       object
 1   BSSID           object
 2   STARTTIMESTAMP  int64 
 3   ENDTIMESTAMP    int64 
dtypes: int64(2), object(2)
memory usage: 254.6+ MB


In [143]:
for n in tqdm(range(len(station_use))):
    station_use['STARTTIMESTAMP'][n] = datetime.fromtimestamp(station_use['STARTTIMESTAMP'][n])
    station_use['ENDTIMESTAMP'][n] = datetime.fromtimestamp(station_use['ENDTIMESTAMP'][n])

  0%|                                                                                      | 0/8342652 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17796\717236959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_use['STARTTIMESTAMP'][n] = datetime.fromtimestamp(station_use['STARTTIMESTAMP'][n])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17796\717236959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_use['ENDTIMESTAMP'][n] = datetime.fromtimestamp(station_use['ENDTIMESTAMP'][n])
100%|█████████████████████████████████████████████████████████████████████| 834

In [144]:
station_use.to_csv('./전처리/wi-fi/station.csv', encoding='utf-8', index=False)

# -------------------------------------------------------------------------------

# Presence_full

In [ ]:
# 파일 분할 저장
txt = './원본/Wi-Fi/

# Presence 읽기
dask_df = dd.read_csv(txt + 'presence_full.txt')

# 분할 저장
dask_df.to_csv('./원본/Wi-Fi/presence full/presence_full-*.csv',encoding='utf-8', index=False)

In [4]:
# 분할한 presence 파일
presence = './원본/Wi-Fi/presence full/'
presence_list = os.listdir(presence)

# pre_df_0 ~ pre_df_166 까지 변수 생성 후
# 해당 변수에 데이터프레임 할당
for n in tqdm(range(len(presence_list))):
    globals()['pre_df_{}'.format(n)] = pd.read_csv(presence + presence_list[n])
    

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [01:59<00:00,  1.39it/s]


In [41]:
print('분할파일 수 :',len(presence_list))

분할파일 수 : 167


In [5]:
pre_df_166

CLIENTMAC         RADIO  STARTTIMESTAMP  ENDTIMESTAMP
0        286D97F3957C  484AE9D99B00      1669463400    1669464000
1        286D97F3A053  D0D3E058F4C0      1669463400    1669464000
2        286D97F46955  CC88C7797860      1669463400    1669464000
3        286D97F6CADE  7C573C74B800      1669463400    1669464000
4        286D97F50312  7C573C7497E0      1669463400    1669464000
...               ...           ...             ...           ...
1310291  C09727778D45  484AE9D2CEA0      1669600200    1669600800
1310292  C0BDC8BE85F7  FC7FF1FBFB50      1669600200    1669600800
1310293  C0D2DD769767  FC7FF1F89B30      1669600200    1669600800
1310294  C45D834C15D5  7C573C746640      1669600200    1669600800
1310295  C4BDE511D05F  7C573C742BE0      1669600200    1669600800

[1310296 rows x 4 columns]

In [11]:
RADIO = ['7C573C74A900','7C573C74B992', '7C573C74BD50', '7C573C7465F0', '7C573C74AE10', '7C573C74F630', '7C573C74D0C0', '7C573C74D0D0', '7C573C7465E0', '7C573C74ECC0', '7C573C74AAF0', '7C573C744DA0', '7C573C74AF32', '7C573C7455B0', '7C573C74B7C0', '7C573C74AFE2', '7C573C74D750', '7C573C743532', '7C573C74D550', '7C573C748D50', '7C573C7470B0', '7C573C7471D2', '7C573C747E60', '7C573C7455A0', '7C573C74A7F0', '7C573C74AAE0', '7C573C74EA70', '7C573C74B360', '7C573C747D10', '7C573C745270', '7C573C74C040', '7C573C74AEE0', '7C573C743420', '7C573C746C80', '7C573C744C10', '7C573C74A940', '7C573C74CE80', '7C573C749C00', '7C573C74B370', '7C573C74AEF0', '7C573C743210', '7C573C74EA60', '7C573C7457B0', '7C573C743030', '7C573C745190', '7C573C74AE60', '7C573C74C3E0', '7C573C74CCC0', '7C573C74A6E0',
 '7C573C745180', '7C573C743520', '7C573C746C90', '7C573C74C3F0', '7C573C74CE92', '7C573C7484F2', '7C573C744C00', '7C573C74B812', '7C573C742BF0', '7C573C74D950', '7C573C743980', '7C573C7497F0', '7C573C74D100', '7C573C74E8D0', '7C573C745F40', '7C573C747400', '7C573C749AB0', '7C573C74BD40', '7C573C74CF00', '7C573C74B7D0', '7C573C74AD70', '7C573C74E280', '7C573C74D940', '7C573C74E8C0', '7C573C7456F0', '7C573C74C050', '7C573C743EE0', '7C573C74ECD0', '7C573C74B2F0', '7C573C749F80', '7C573C74EF30', '7C573C74F780', 'D015A6FF5060', '7C573C7454E0', '7C573C74E550', '7C573C74D6A0', '7C573C74E540', '7C573C74E290', '7C573C7454F0', 'D015A6FF4F30', '7C573C745BA0', '7C573C743020', '7C573C74AC00', '7C573C74D740', '7C573C74F1F0', '7C573C742BE0', '7C573C743DF0', '7C573C7B76A0', '7C573C74AC10', '7C573C7457A0', '7C573C74CF12', '7C573C74A950', '7C573C7B76B0', 'D015A6FF5070', '7C573C74EF20',
 '7C573C745BB0', '7C573C74B2E0', '7C573C74B800', '7C573C748D40', '7C573C74A7E0', '7C573C749C10', '7C573C743200', '7C573C743430', '7C573C74A6F0', '7C573C74AE00', '7C573C744DB0', '7C573C74BDB0', '7C573C74F1E0', '7C573C749AA0', '7C573C7497E0', '7C573C743EF0', '7C573C74CCD0', '7C573C74D540', '7C573C745F50', '7C573C74F890', '7C573C74AD60', '7C573C74BDA0', '7C573C743990', '7C573C74F790', '7C573C747E70', '7C573C74A910', '7C573C74AFF2', '7C573C74AE70', '7C573C74B980', '7C573C745260', '7C573C74D6B0', '7C573C7470A0', '7C573C747410', 'D015A6FF4F20', '7C573C749F90', '7C573C74AF22', '7C573C7471C0', '7C573C7456E0',
 '7C573C7484E0', '7C573C74D110', '7C573C74F880', '7C573C747D00', '7C573C743DE0', '7C573C74F620']

In [17]:
RADIO_list = None
for n in range(len(RADIO)):
    if n == 0:
        RADIO_list = RADIO[n]
    else:
        RADIO_list += ('|'+ RADIO[n])

In [22]:
for n in tqdm(range(len(presence_list))):
    a = globals()['pre_df_{}'.format(n)]
    globals()['pre_df_{}_use'.format(n)] = a[a['RADIO'].str.contains(RADIO_list)]
    globals()['pre_df_{}_use'.format(n)].reset_index(drop=True, inplace=True)
    globals()['pre_df_{}_use'.format(n)].to_csv(f'./전처리/wi-fi/presence_full/pre_df_{n}.csv', encoding='utf-8', index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [12:22<00:00,  4.45s/it]


In [24]:
presence_sum1 = 0
presence_sum2 = 0
for n in tqdm(range(len(presence_list))):
    presence_sum1 += len(globals()['pre_df_{}'.format(n)])
    presence_sum2 += len(globals()['pre_df_{}_use'.format(n)])

print('원본 : ', presence_sum1)    
print('추출 : ', presence_sum2)
print('비율 : ', presence_sum2 / presence_sum1 * 100)

100%|██████████████████████████████████████████████████████████████████████████████| 167/167 [00:00<00:00, 6680.10it/s]

원본 :  218821529
추출 :  118980238
비율 :  54.37318647014846


원본 :  218,821,529         [2억1천8백만 건] <br/>
추출 :  118,980,238         [1억1천8백만 건]<br/>
비율 :  54.37318647014846

# Presence_full - Unix time 변환

In [7]:
presence = './전처리/wi-fi/presence_full/'
presence_list = os.listdir(presence)

for n in tqdm(range(len(presence_list))):
    globals()['pre_df_{}_use'.format(n)] = pd.read_csv(presence + presence_list[n])
    df = globals()['pre_df_{}_use'.format(n)]
    
    for i in range(len(df)):
        df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
        df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
    
    df.to_csv(f'./전처리/wi-fi/presence_full_Unix변환/pre_df_{n}.csv', encoding='utf-8', index=False)

  0%|                                                                                          | 0/167 [00:00<?, ?it/s]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
  1%|▍                                                                               | 1/167 [00:34<1:34:44, 34.

  6%|████▊                                                                            | 10/167 [03:37<48:40, 18.60s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
  7%|█████▎                                                                           | 11/167 [03:55<47:44, 18.

 12%|█████████▋                                                                       | 20/167 [06:39<43:21, 17.70s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 13%|██████████▏                                                                      | 21/167 [06:56<42:38, 17.

 18%|██████████████▌                                                                  | 30/167 [09:26<36:08, 15.83s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 19%|███████████████                                                                  | 31/167 [09:42<35:38, 15.

 24%|███████████████████▍                                                             | 40/167 [12:06<33:04, 15.63s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 25%|███████████████████▉                                                             | 41/167 [12:21<32:30, 15.

...

 72%|█████████████████████████████████████████████████████████▍                      | 120/167 [39:02<17:57, 22.92s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 72%|█████████████████████████████████████████████████████████▉                      | 121/167 [39:25<17:23, 22.

 78%|██████████████████████████████████████████████████████████████▎                 | 130/167 [42:51<13:23, 21.72s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 78%|██████████████████████████████████████████████████████████████▊                 | 131/167 [43:13<13:02, 21.

 84%|███████████████████████████████████████████████████████████████████             | 140/167 [46:19<08:55, 19.85s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 84%|███████████████████████████████████████████████████████████████████▌            | 141/167 [46:37<08:22, 19.

 90%|███████████████████████████████████████████████████████████████████████▊        | 150/167 [49:37<05:32, 19.54s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 90%|████████████████████████████████████████████████████████████████████████▎       | 151/167 [49:55<05:05, 19.

 96%|████████████████████████████████████████████████████████████████████████████▋   | 160/167 [52:39<02:04, 17.84s/it]C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['STARTTIMESTAMP'][i] = datetime.fromtimestamp(df['STARTTIMESTAMP'][i])
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11976\2013097779.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ENDTIMESTAMP'][i] = datetime.fromtimestamp(df['ENDTIMESTAMP'][i])
 96%|█████████████████████████████████████████████████████████████████████████████▏  | 161/167 [52:55<01:44, 17.

# -------------------------------------------------------------------------------

# 특정 기간 추출

### Station

In [8]:
station = pd.read_csv('./전처리/wi-fi/station.csv', encoding='utf-8')

In [10]:
station.head()

CLIENTMAC         BSSID       STARTTIMESTAMP         ENDTIMESTAMP
0  940C98B56271  7C573C747400  2020-04-29 15:20:00  2020-04-29 15:30:00
1  9476B78E5BB3  7C573C746C80  2020-04-29 15:20:00  2020-04-29 15:30:00
2  948BC111791C  7C573C7457B0  2020-04-29 15:20:00  2020-04-29 15:30:00
3  948BC1006C66  7C573C743030  2020-04-29 15:20:00  2020-04-29 15:30:00
4  9476B7B12657  7C573C74B7C0  2020-04-29 15:20:00  2020-04-29 15:30:00

In [11]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8342652 entries, 0 to 8342651
Data columns (total 4 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   CLIENTMAC       object
 1   BSSID           object
 2   STARTTIMESTAMP  object
 3   ENDTIMESTAMP    object
dtypes: object(4)
memory usage: 254.6+ MB


In [19]:
station['STARTTIMESTAMP'] = pd.to_datetime(station['STARTTIMESTAMP'])

In [21]:
station['ENDTIMESTAMP'] = pd.to_datetime(station['ENDTIMESTAMP'])

In [22]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8342652 entries, 0 to 8342651
Data columns (total 4 columns):
 #   Column          Dtype         
---  ------          -----         
 0   CLIENTMAC       object        
 1   BSSID           object        
 2   STARTTIMESTAMP  datetime64[ns]
 3   ENDTIMESTAMP    datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 254.6+ MB


In [29]:
station['USE_TIME'] = station['ENDTIMESTAMP'] - station['STARTTIMESTAMP']

In [30]:
station

CLIENTMAC         BSSID      STARTTIMESTAMP        ENDTIMESTAMP  \
0        940C98B56271  7C573C747400 2020-04-29 15:20:00 2020-04-29 15:30:00   
1        9476B78E5BB3  7C573C746C80 2020-04-29 15:20:00 2020-04-29 15:30:00   
2        948BC111791C  7C573C7457B0 2020-04-29 15:20:00 2020-04-29 15:30:00   
3        948BC1006C66  7C573C743030 2020-04-29 15:20:00 2020-04-29 15:30:00   
4        9476B7B12657  7C573C74B7C0 2020-04-29 15:20:00 2020-04-29 15:30:00   
...               ...           ...                 ...                 ...   
8342647  A674B0AB737C  7C573C7455A0 2022-11-27 00:10:00 2022-11-27 00:20:00   
8342648  8A4610BA5CC4  7C573C7497F0 2022-11-27 00:10:00 2022-11-27 00:20:00   
8342649  6E4DE27A1B9B  7C573C7470B0 2022-11-27 00:10:00 2022-11-27 00:20:00   
8342650  04B1679C7257  7C573C74B2E0 2022-11-27 00:10:00 2022-11-27 00:20:00   
8342651  2418C627ADD7  7C573C7454F0 2022-11-27 00:10:00 2022-11-27 00:20:00   

               USE_TIME  
0       0 days 00:10:00  
1       0 days 00:10:00  
2       0 days 00:10:00  
3       0 days 00:10:00  
4       0 days 00:10:00  
...                 ...  
8342647 0 days 00:10:00  
8342648 0 days 00:10:00  
8342649 0 days 00:10:00  
8342650 0 days 00:10:00  
8342651 0 days 00:10:00  

[8342652 rows x 5 columns]

In [42]:
station = station[(station['STARTTIMESTAMP'] >= datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')) &
                    (station['ENDTIMESTAMP'] < datetime.strptime('2022-11-01 00:00:00', '%Y-%m-%d %H:%M:%S'))]
station

CLIENTMAC         BSSID      STARTTIMESTAMP        ENDTIMESTAMP  \
2143588  C6D6DEC1D5AE  7C573C743990 2021-01-03 00:30:00 2021-01-03 00:40:00   
2143589  C8D7B0FA6430  7C573C74A7E0 2021-01-03 00:30:00 2021-01-03 00:40:00   
2143590  CC50E30D9E39  7C573C745BA0 2021-01-03 00:30:00 2021-01-03 00:40:00   
2143591  CC50E33A4B69  7C573C743420 2021-01-03 00:30:00 2021-01-03 00:40:00   
2143592  CC50E33A4B7C  7C573C74D0C0 2021-01-03 00:30:00 2021-01-03 00:40:00   
...               ...           ...                 ...                 ...   
8204470  EE47B2383963  7C573C747E70 2022-10-31 23:10:00 2022-10-31 23:20:00   
8204471  EAD4464BA405  7C573C744DB0 2022-10-31 23:10:00 2022-10-31 23:20:00   
8204472  F254AF6F0D31  7C573C7454F0 2022-10-31 23:10:00 2022-10-31 23:20:00   
8204473  FA13A1EF2B97  7C573C744DB0 2022-10-31 23:10:00 2022-10-31 23:20:00   
8204474  F69A28E9916A  7C573C7465F0 2022-10-31 23:10:00 2022-10-31 23:20:00   

               USE_TIME  
2143588 0 days 00:10:00  
2143589 0 days 00:10:00  
2143590 0 days 00:10:00  
2143591 0 days 00:10:00  
2143592 0 days 00:10:00  
...                 ...  
8204470 0 days 00:10:00  
8204471 0 days 00:10:00  
8204472 0 days 00:10:00  
8204473 0 days 00:10:00  
8204474 0 days 00:10:00  

[5896413 rows x 5 columns]

In [43]:
station.to_csv('./전처리/wi-fi/station.csv', encoding='utf-8', index=False)

In [31]:
station.value_counts('USE_TIME')

USE_TIME
0 days 00:10:00    8342652
dtype: int64

### Presence full (필요 기간 추출)

In [44]:
presence = './전처리/wi-fi/presence_full_Unix변환/'
presence_list = os.listdir(presence)
                           
for n in tqdm(range(len(presence_list))):
    df = pd.read_csv(presence + presence_list[n])
    
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])
    df['USE_TIME'] = df['ENDTIMESTAMP'] - df['STARTTIMESTAMP']
    
    df = df[(df['STARTTIMESTAMP'] >= datetime.strptime('2021-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')) &
            (df['ENDTIMESTAMP'] < datetime.strptime('2022-11-01 00:00:00', '%Y-%m-%d %H:%M:%S'))]
    
    globals()['pre_df_{}_time'.format(n)] = df
    
    df.to_csv(f'./전처리/wi-fi/presence_full_time/pre_df_time_{n}.csv', encoding='utf-8', index=False)

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [16:54<00:00,  6.07s/it]


In [45]:
presence_sum = 0
for n in tqdm(range(len(presence_list))):
    presence_sum += len(globals()['pre_df_{}_time'.format(n)])
print(presence_sum)

100%|████████████████████████████████████████████████████████████████████████████| 167/167 [00:00<00:00, 167251.38it/s]

73253510


In [2]:
presence = './전처리/wi-fi/presence_full_time/'
presence_list = os.listdir(presence)
                           
for n in tqdm(range(len(presence_list))):
    globals()['pre_df_{}_time'.format(n)] = pd.read_csv(presence + presence_list[n])

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:52<00:00,  3.16it/s]


In [3]:
pre_df_time = []
for n in range(len(presence_list)):
    pre_df_time.append('pre_df_{}_time'.format(n))

In [6]:
','.join(pre_df_time)

'pre_df_0_time,pre_df_1_time,pre_df_2_time,pre_df_3_time,pre_df_4_time,pre_df_5_time,pre_df_6_time,pre_df_7_time,pre_df_8_time,pre_df_9_time,pre_df_10_time,pre_df_11_time,pre_df_12_time,pre_df_13_time,pre_df_14_time,pre_df_15_time,pre_df_16_time,pre_df_17_time,pre_df_18_time,pre_df_19_time,pre_df_20_time,pre_df_21_time,pre_df_22_time,pre_df_23_time,pre_df_24_time,pre_df_25_time,pre_df_26_time,pre_df_27_time,pre_df_28_time,pre_df_29_time,pre_df_30_time,pre_df_31_time,pre_df_32_time,pre_df_33_time,pre_df_34_time,pre_df_35_time,pre_df_36_time,pre_df_37_time,pre_df_38_time,pre_df_39_time,pre_df_40_time,pre_df_41_time,pre_df_42_time,pre_df_43_time,pre_df_44_time,pre_df_45_time,pre_df_46_time,pre_df_47_time,pre_df_48_time,pre_df_49_time,pre_df_50_time,pre_df_51_time,pre_df_52_time,pre_df_53_time,pre_df_54_time,pre_df_55_time,pre_df_56_time,pre_df_57_time,pre_df_58_time,pre_df_59_time,pre_df_60_time,pre_df_61_time,pre_df_62_time,pre_df_63_time,pre_df_64_time,pre_df_65_time,pre_df_66_time,pre_

In [10]:
df = pd.concat([pre_df_0_time,pre_df_1_time,pre_df_2_time,pre_df_3_time,pre_df_4_time,pre_df_5_time,pre_df_6_time,pre_df_7_time,pre_df_8_time,pre_df_9_time,pre_df_10_time,pre_df_11_time,pre_df_12_time,pre_df_13_time,pre_df_14_time,pre_df_15_time,pre_df_16_time,pre_df_17_time,pre_df_18_time,pre_df_19_time,pre_df_20_time,pre_df_21_time,pre_df_22_time,pre_df_23_time,pre_df_24_time,pre_df_25_time,pre_df_26_time,pre_df_27_time,pre_df_28_time,pre_df_29_time,pre_df_30_time,pre_df_31_time,pre_df_32_time,pre_df_33_time,pre_df_34_time,pre_df_35_time,pre_df_36_time,pre_df_37_time,pre_df_38_time,pre_df_39_time,pre_df_40_time,pre_df_41_time,pre_df_42_time,pre_df_43_time,pre_df_44_time,pre_df_45_time,pre_df_46_time,pre_df_47_time,pre_df_48_time,pre_df_49_time,pre_df_50_time,pre_df_51_time,pre_df_52_time,pre_df_53_time,pre_df_54_time,pre_df_55_time,pre_df_56_time,pre_df_57_time,pre_df_58_time,pre_df_59_time,pre_df_60_time,pre_df_61_time,pre_df_62_time,pre_df_63_time,pre_df_64_time,pre_df_65_time,pre_df_66_time,pre_df_67_time,pre_df_68_time,pre_df_69_time,pre_df_70_time,pre_df_71_time,pre_df_72_time,pre_df_73_time,pre_df_74_time,pre_df_75_time,pre_df_76_time,pre_df_77_time,pre_df_78_time,pre_df_79_time,pre_df_80_time,pre_df_81_time,pre_df_82_time,pre_df_83_time,pre_df_84_time,pre_df_85_time,pre_df_86_time,pre_df_87_time,pre_df_88_time,pre_df_89_time,pre_df_90_time,pre_df_91_time,pre_df_92_time,pre_df_93_time,pre_df_94_time,pre_df_95_time,pre_df_96_time,pre_df_97_time,pre_df_98_time,pre_df_99_time,pre_df_100_time,pre_df_101_time,pre_df_102_time,pre_df_103_time,pre_df_104_time,pre_df_105_time,pre_df_106_time,pre_df_107_time,pre_df_108_time,pre_df_109_time,pre_df_110_time,pre_df_111_time,pre_df_112_time,pre_df_113_time,pre_df_114_time,pre_df_115_time,pre_df_116_time,pre_df_117_time,pre_df_118_time,pre_df_119_time,pre_df_120_time,pre_df_121_time,pre_df_122_time,pre_df_123_time,pre_df_124_time,pre_df_125_time,pre_df_126_time,pre_df_127_time,pre_df_128_time,pre_df_129_time,pre_df_130_time,pre_df_131_time,pre_df_132_time,pre_df_133_time,pre_df_134_time,pre_df_135_time,pre_df_136_time,pre_df_137_time,pre_df_138_time,pre_df_139_time,pre_df_140_time,pre_df_141_time,pre_df_142_time,pre_df_143_time,pre_df_144_time,pre_df_145_time,pre_df_146_time,pre_df_147_time,pre_df_148_time,pre_df_149_time,pre_df_150_time,pre_df_151_time,pre_df_152_time,pre_df_153_time,pre_df_154_time,pre_df_155_time,pre_df_156_time,pre_df_157_time,pre_df_158_time,pre_df_159_time,pre_df_160_time,pre_df_161_time,pre_df_162_time,pre_df_163_time,pre_df_164_time,pre_df_165_time,pre_df_166_time], axis=0)

In [14]:
df.sort_values(by=['CLIENTMAC', 'STARTTIMESTAMP'], ascending=[True, True], inplace=True)

In [15]:
df.reset_index(drop=True, inplace=True)

In [16]:
df

CLIENTMAC         RADIO       STARTTIMESTAMP  \
0            000000000  7C573C74B360  2021-01-01 12:20:00   
1            000000000  7C573C74A6F0  2021-01-02 16:40:00   
2            000000000  7C573C74E550  2021-01-02 18:30:00   
3            000000000  7C573C745BB0  2021-01-02 22:20:00   
4            000000000  7C573C7465F0  2021-01-03 14:10:00   
...                ...           ...                  ...   
73253505  FDFFED481278  7C573C7457B0  2021-02-06 12:20:00   
73253506  FDFFF4B2065E  7C573C7454F0  2021-01-21 17:30:00   
73253507  FDFFF508017C  7C573C747E70  2021-12-05 01:10:00   
73253508  FDFFF5E3528A  7C573C7455A0  2021-01-01 15:50:00   
73253509  FDFFF697FB4F  7C573C74D100  2021-02-14 18:20:00   

                 ENDTIMESTAMP         USE_TIME  
0         2021-01-01 12:30:00  0 days 00:10:00  
1         2021-01-02 16:50:00  0 days 00:10:00  
2         2021-01-02 18:40:00  0 days 00:10:00  
3         2021-01-02 22:30:00  0 days 00:10:00  
4         2021-01-03 14:20:00  0 days 00:10:00  
...                       ...              ...  
73253505  2021-02-06 12:30:00  0 days 00:10:00  
73253506  2021-01-21 17:40:00  0 days 00:10:00  
73253507  2021-12-05 01:20:00  0 days 00:10:00  
73253508  2021-01-01 16:00:00  0 days 00:10:00  
73253509  2021-02-14 18:30:00  0 days 00:10:00  

[73253510 rows x 5 columns]

In [17]:
df.to_csv('./전처리/wi-fi/presence.csv', encoding='utf-8', index=False)

# 사용량이 많은 Users

In [41]:
station = pd.read_csv('./전처리/wi-fi/station.csv', encoding='utf-8')

In [44]:
station.head()

CLIENTMAC         BSSID       STARTTIMESTAMP         ENDTIMESTAMP  \
0  C6D6DEC1D5AE  7C573C743990  2021-01-03 00:30:00  2021-01-03 00:40:00   
1  C8D7B0FA6430  7C573C74A7E0  2021-01-03 00:30:00  2021-01-03 00:40:00   
2  CC50E30D9E39  7C573C745BA0  2021-01-03 00:30:00  2021-01-03 00:40:00   
3  CC50E33A4B69  7C573C743420  2021-01-03 00:30:00  2021-01-03 00:40:00   
4  CC50E33A4B7C  7C573C74D0C0  2021-01-03 00:30:00  2021-01-03 00:40:00   

          USE_TIME  
0  0 days 00:10:00  
1  0 days 00:10:00  
2  0 days 00:10:00  
3  0 days 00:10:00  
4  0 days 00:10:00

In [45]:
station.value_counts('CLIENTMAC')

CLIENTMAC
B4E62D460D18    67138
B4E62D460C9E    67132
CC50E33A4B72    67131
CC50E30D9E39    67131
CC50E33A4B79    67128
                ...  
5A4ACC7FCC93        1
5A4B07B88AFB        1
C28035CEE011        1
5A4C0DDEBF4E        1
000003220846        1
Length: 78131, dtype: int64

In [46]:
station_user = pd.DataFrame(station.value_counts('CLIENTMAC'))

In [48]:
station_user.reset_index(inplace=True)
station_user_TOP1000 = station_user.head(1000)

In [51]:
station_user_TOP1000.rename(columns={0:'STATION_USER'},inplace=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_18376\2504388280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_user_TOP1000.rename(columns={0:'STATION_USER'},inplace=True)


In [52]:
station_user_TOP1000

CLIENTMAC  STATION_USER
0    B4E62D460D18         67138
1    B4E62D460C9E         67132
2    CC50E33A4B72         67131
3    CC50E30D9E39         67131
4    CC50E33A4B79         67128
..            ...           ...
995  CAE0831230EF           957
996  266F283A6337           956
997  CE523ABE5785           956
998  30FCEB150975           955
999  F4428F2D0B2D           954

[1000 rows x 2 columns]

In [62]:
station_user_TOP1000.to_csv('./전처리/wi-fi/USERS_TOP1000/station_user_TOP1000.csv',encoding='utf-8')

In [53]:
presence_user = pd.DataFrame(df.value_counts('CLIENTMAC'))
presence_user.reset_index(inplace=True)

In [54]:
presence_user.head()

CLIENTMAC      0
0  88571D052EB0  66907
1  808C97A82C32  66900
2  CC50E33A4C60  66857
3  CC50E33A4BFB  66835
4  CC50E33A4B79  66830

In [55]:
presence_user.rename(columns={0:'PRESENCE_USER'},inplace=True)

In [58]:
presence_user.head()

CLIENTMAC  PRESENCE_USER
0  88571D052EB0          66907
1  808C97A82C32          66900
2  CC50E33A4C60          66857
3  CC50E33A4BFB          66835
4  CC50E33A4B79          66830

In [59]:
presence_user_TOP1000 = presence_user.head(1000)

In [60]:
presence_user_TOP1000

CLIENTMAC  PRESENCE_USER
0    88571D052EB0          66907
1    808C97A82C32          66900
2    CC50E33A4C60          66857
3    CC50E33A4BFB          66835
4    CC50E33A4B79          66830
..            ...            ...
995  C4D9876B28C6           9072
996  286D978ED53A           9069
997  0008DC5DA506           9059
998  E4FAED896168           9055
999  948BC1101040           9038

[1000 rows x 2 columns]

In [61]:
presence_user_TOP1000.to_csv('./전처리/wi-fi/USERS_TOP1000/presence_user_TOP1000.csv',encoding='utf-8')

# 할당된 변수 지우기
### (메모리 초기화)

In [77]:
import gc

In [81]:
gc.collect()

0

In [85]:
# 메모리 초기화
%reset -f

In [87]:
# 변수확인
%whos

Interactive namespace is empty.
